## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split

## Data Preprocessing

In [ ]:
#reading the csv
retail_csv = pd.read_csv("data/Online Retail Data Set.csv", encoding='unicode_escape')
retail_csv.head()

In [ ]:
#checking the data type, shape and overall description of the dataset 
retail_csv.info(), retail_csv.shape, retail_csv.describe(include='all')

In [ ]:
#checking missing value per column
retail_csv.isnull().sum()

So, we can notice that:
- there are missing values in `Description` and `CustomerID` 
- `InvoiceDate` needs to be converted to date-time object
- `UnitPrice` and `Quantity` have negative values
- `CustomerID` can be a categorial variable

In [ ]:
#Handling the missing values first by droping rows where Description or CustomerID is missing
retail_csv = retail_csv.dropna(subset=['Description','CustomerID'])
retail_csv.isnull().sum(), retail_csv.shape

In [ ]:
#Convert InvoiceDate column to date-time object
retail_csv['InvoiceDate'] = pd.to_datetime(retail_csv['InvoiceDate'], dayfirst='True')
retail_csv.info()

In [ ]:
#Dealing with the negative value from UnitPrice and Quantity
retail_csv = retail_csv[(retail_csv['Quantity'] > 0) & (retail_csv['UnitPrice'] > 0)]
retail_csv.describe(include="all")

In [ ]:
retail_csv['InvoiceNo'].value_counts(), retail_csv['StockCode'].value_counts(), retail_csv['CustomerID'].value_counts(), retail_csv['Country'].value_counts()

Now, we will encode `InvoiceNo`, `StockCode`, `CustomerID`, `Country` as categorial variable

In [ ]:
#Encoding the categorial variables
categorical_columns = ['InvoiceNo', 'StockCode', 'Description', 'Country','CustomerID']
label_encoders = {col:LabelEncoder() for col in categorical_columns}

for col in categorical_columns:
    retail_csv[col] = label_encoders[col].fit_transform(retail_csv[col])
    
#Feature scaling for UnitPrice and Quantity
# scaler = StandardScaler()
# retail_csv[['Quantity','UnitPrice']] = scaler.fit_transform(retail_csv[['Quantity','UnitPrice']])

#check the new data
retail_csv.head(10)

## Exploratory Data Analysis

In [ ]:
#Let's see the summary of our preprocessed data
retail_summary = retail_csv.describe()
retail_null_check = retail_csv.isnull().sum()
print(retail_summary)
print(f"\nChecking for missing values in each column: \n{retail_null_check}")

Some observation:
- `Quantity` and `UnitPrice` are properly scaled and has no negative value. but both have some outliers
- The categorial columns (`InvoiceNo`, `StockCode`, `Description`, `Country`) are encoded properly.
- has no missing values

In [ ]:
#Visualizing outliers of UnitPrice
plt.figure(figsize=(10, 6))
sns.boxplot(x=retail_csv['UnitPrice'])
plt.title('Boxplot of UnitPrice')
plt.xlabel('UnitPrice')
plt.show()

In [ ]:
#Visualizing outliers of Quantity
plt.figure(figsize=(10, 6))
sns.boxplot(x=retail_csv['Quantity'])
plt.title('Boxplot of Quantity')
plt.xlabel('UnitPrice')
plt.show()

In [ ]:
# retail_csv.to_csv("preprocessed_retail.csv",index = "False")

In [ ]:
unitprice_percentile = retail_csv['UnitPrice'].quantile(0.73)
quantity_percentile = retail_csv['Quantity'].quantile(0.73)

retail_filtered = retail_csv[
    (retail_csv['UnitPrice'] <= unitprice_percentile) &
    (retail_csv['Quantity'] <= quantity_percentile)
]

retail_filtered.describe()

In [ ]:
#Visualizing outliers of UnitPrice
plt.figure(figsize=(10, 6))
sns.boxplot(x=retail_filtered['UnitPrice'])
plt.title('Boxplot of UnitPrice')
plt.xlabel('UnitPrice')
plt.show()

In [ ]:
#Visualizing outliers of Quantity
plt.figure(figsize=(10, 6))
sns.boxplot(x=retail_filtered['Quantity'])
plt.title('Boxplot of Quantity')
plt.xlabel('UnitPrice')
plt.show()

In [ ]:
#checking the dataframe after filtering
retail_filtered.head()

We don't need `InvoiceNo`,`CustomerID`,`Description` columns for the prediction. So, We will drop them

In [ ]:
retail_filtered = retail_filtered.drop(columns=["InvoiceNo","Description"])
retail_filtered.head()

In [ ]:
# Histograms for all numerical features
numerical_features = retail_filtered.select_dtypes(include=['int64', 'float64']).columns
for feature in numerical_features:
    plt.figure(figsize=(10, 5))
    sns.histplot(retail_filtered[feature], kde=True)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
# Scatter plot for 'UnitPrice' vs 'Quantity'
plt.figure(figsize=(10, 5))
sns.scatterplot(x='UnitPrice', y='Quantity', data=retail_filtered)
plt.title('UnitPrice vs Quantity Scatter Plot')
plt.xlabel('UnitPrice')
plt.ylabel('Quantity')
plt.show()

In [ ]:
# # Correlation heatmap
# correlation_matrix = retail_filtered.corr()
# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Correlation Heatmap')
# plt.show()

In [ ]:
# class Node:
#     def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
#         self.feature = feature
#         self.threshold = threshold
#         self.left = left
#         self.right = right
#         self.value = value
        
#     def is_leaf_node(self):
#         return self.value is not None


# class DecisionTree:
#     def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
#         self.min_samples_split=min_samples_split
#         self.max_depth=max_depth
#         self.n_features=n_features
#         self.root=None

#     def fit(self, X, y):
#         self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
#         self.root = self._grow_tree(X, y)

#     def _grow_tree(self, X, y, depth=0):
#         n_samples, n_feats = X.shape
#         n_labels = len(np.unique(y))

#         # check the stopping criteria
#         if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
#             leaf_value = self._most_common_label(y)
#             return Node(value=leaf_value)

#         feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

#         # find the best split
#         best_feature, best_thresh = self._best_split(X, y, feat_idxs)

#         # create child nodes
#         left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
#         left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
#         right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
#         return Node(best_feature, best_thresh, left, right)


#     def _best_split(self, X, y, feat_idxs):
#         best_gain = -1
#         split_idx, split_threshold = None, None

#         for feat_idx in feat_idxs:
#             X_column = X[:, feat_idx]
#             thresholds = np.unique(X_column)

#             for thr in thresholds:
#                 # calculate the information gain
#                 gain = self._information_gain(y, X_column, thr)

#                 if gain > best_gain:
#                     best_gain = gain
#                     split_idx = feat_idx
#                     split_threshold = thr

#         return split_idx, split_threshold


#     def _information_gain(self, y, X_column, threshold):
#         # parent entropy
#         parent_entropy = self._entropy(y)

#         # create children
#         left_idxs, right_idxs = self._split(X_column, threshold)

#         if len(left_idxs) == 0 or len(right_idxs) == 0:
#             return 0
        
#         # calculate the weighted avg. entropy of children
#         n = len(y)
#         n_l, n_r = len(left_idxs), len(right_idxs)
#         e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
#         child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

#         # calculate the IG
#         information_gain = parent_entropy - child_entropy
#         return information_gain

#     def _split(self, X_column, split_thresh):
#         left_idxs = np.argwhere(X_column <= split_thresh).flatten()
#         right_idxs = np.argwhere(X_column > split_thresh).flatten()
#         return left_idxs, right_idxs

#     def _entropy(self, y):
#         hist = np.bincount(y)
#         ps = hist / len(y)
#         return -np.sum([p * np.log(p) for p in ps if p>0])


#     def _most_common_label(self, y):
#         counter = Counter(y)
#         value = counter.most_common(1)[0][0]
#         return value

#     def predict(self, X):
#         return np.array([self._traverse_tree(x, self.root) for x in X])

#     def _traverse_tree(self, x, node):
#         if node.is_leaf_node():
#             return node.value

#         if x[node.feature] <= node.threshold:
#             return self._traverse_tree(x, node.left)
#         return self._traverse_tree(x, node.right)

In [ ]:
# class RandomForestClassifier():
    
#     def __init__(self, n_trees = 10, max_depth =10, min_samples_split = 2, n_feature = None):
#         self.n_trees = n_trees
#         self.max_depth = max_depth
#         self.min_sample_split = min_samples_split
#         self.n_feature = n_feature
#         self.trees = []
        
#     def fit(self, X, y):
#         self.trees = []
#         for _ in range(self.n_trees):
#             tree = DecisionTree(max_depth=self.max_depth,
#                                 min_samples_split = self.min_sample_split,
#                                 n_features = self.n_feature)
#             X_sample, y_sample = self._bootstrap_samples(X,y)
#             tree.fit(X_sample, y_sample)
#             self.trees.append(tree)
            
#     # def predict(self, X):
#     #     for tree in self.trees:
#     #         predictions =np.array(tree.predict(X))
#     #     tree_preds = np.swapaxes(predictions, 0,1)
#     #     predictions = np.array([self._common_label(pred) for pred in tree_preds])
#     #     return predictions
    
#     def predict(self, X):
#         predictions = np.array([tree.predict(X) for tree in self.trees])
#         tree_preds = np.swapaxes(predictions, 0, 1)
#         predictions = np.array([self._common_label(pred) for pred in tree_preds])
#         return predictions
    
#     def _bootstrap_samples(self, X, y):
#         n_samples = X.shape[0]
#         idx = np.random.choice(n_samples, n_samples, replace = True)
#         return X[idx],y[idx]
    
#     def _common_label(self, y):
#         counter = Counter()
#         most_common = counter.most_common(1)[0][0]
#         return most_common

## Splitting data

In [ ]:
# retail_filtered.to_csv("retail_filtered.csv", index=False)

In [ ]:
X = retail_filtered.drop(['Quantity','InvoiceDate'], axis =1).values
y = retail_filtered['Quantity'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)


In [ ]:
X_train

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rfr = RandomForestRegressor(n_estimators=1000)
# rfr.fit(X_train,y_train)

In [ ]:
# y_pred = rfr.predict(X_test)

## Building Model

Now we'll be making a random forest regressor from scratch

In [ ]:
import numpy as np

# Decision Node used in the Decision Tree
class DecisionNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

# Decision Tree Regressor
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=float('inf')):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        if num_samples >= self.min_samples_split and depth < self.max_depth:
            best_feat, best_thresh = self._best_split(X, y, num_features)
            if best_feat is not None:
                indices_left = X[:, best_feat] < best_thresh
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                left = self._grow_tree(X_left, y_left, depth + 1)
                right = self._grow_tree(X_right, y_right, depth + 1)
                return DecisionNode(feature_index=best_feat, threshold=best_thresh, left=left, right=right)
        return DecisionNode(value=np.mean(y))

    def _best_split(self, X, y, num_features):
        best_feat, best_thresh = None, None
        best_mse = float('inf')
        current_value = np.mean(y)
        current_mse = np.mean((y - current_value) ** 2)
        for feat in range(num_features):
            thresholds = np.unique(X[:, feat])
            for thresh in thresholds:
                indices_left = X[:, feat] < thresh
                y_left, y_right = y[indices_left], y[~indices_left]
                if len(y_left) > 0 and len(y_right) > 0:
                    mse = (len(y_left) * np.mean((y_left - np.mean(y_left)) ** 2) + len(y_right) * np.mean((y_right - np.mean(y_right)) ** 2)) / len(y)
                    if mse < best_mse:
                        best_mse = mse
                        best_feat = feat
                        best_thresh = thresh
        return best_feat, best_thresh

    def predict(self, X):
        return np.array([self._predict(inputs, self.root) for inputs in X])
    
    def _predict(self, inputs, node):
        if node.value is not None:
            return node.value
        if inputs[node.feature_index] < node.threshold:
            return self._predict(inputs, node.left)
        return self._predict(inputs, node.right)

# Random Forest Regressor
class RandomForestRegressor:
    def __init__(self, n_estimators=5, min_samples_split=2, max_depth=float('inf')):
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(self.min_samples_split, self.max_depth)
            bootstrap_indices = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
            tree.fit(X[bootstrap_indices], y[bootstrap_indices])
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)


## Training and Evaluation

In [ ]:
regressor = RandomForestRegressor()

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

## Result Visualization

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Visualization
plt.figure(figsize=(10, 5))

# Scatter plot of actual vs predicted values
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--k')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('True vs Predicted Values')

# Text box for metrics
textstr = f'MSE: {mse:.2f}\nR2: {r2:.2f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, fontsize=14, verticalalignment='top', bbox=props)

plt.show()